<a href="https://colab.research.google.com/github/ErrorOfTheGeneration/pyBotTG_google/blob/main/pyBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random   # для рандомных ответов из словаря ответов
import nltk     # для работы с опечатками

In [13]:
import random
# словарь словарей. Нужен нам для того, чтобы определять, что хочет пользователь
# и соттветствующе отвечать ему
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['Привет!', 'Хай', 'Добрый день'], # примеры приветствия
            'resposes': ['Доброго времени суток', 'Прив']  #ответы бота
        },
        'bye': {
            'examples': ['Пока', 'До свидания', 'Увидимся'],
            'resposes': ['Счастливо',  'Удачного дня', 'Приходите ещё']
        }
    }
}

In [14]:


def clean(text): #принимает текст, возвращает чищенный текст ниж.регистр и без знаков препинания
  return ''.join([simbol for simbol in text.lower() if simbol in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '])

# функция match нужна для того, чтобы бот не был чувствителен к регистру или знакам препинания

# nltk позволяет вычислять расстояние Ливенштейна 
# между строками, т.е. кол-во редакторских исправлений, 
# чтобы из одного слова получить другое
def match(example, text):      
  return nltk.edit_distance(clean(text),clean(example)) / len(example) < 0.4

# пробегаем по всему нашему словарю и ищем, в какой интент (намерение) попадает
# введённое пользователем слово
def get_intent(text):
  for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
      if match(example, text):
        return random.choice(value['resposes'])
  return'Я не понимаю, чего от меня хотят'

In [ ]:
question = input()            # просим пользователя ввести
answer = get_intent(question)
print(answer)

In [17]:
while question != 'выход':
  question = input()            
  answer = get_intent(question)
  print(answer)

выход
Я не понимаю, чего от меня хотят
